In [1]:
import psycopg2

# Replace {your_password} with the actual password
password = "4feehGQ4diL3S45mAbhJleSihTcq"


In [2]:

def executeSQL(sql_queue,database): 
    # Establish a connection to the PostgreSQL server with AUTOCOMMIT enabled
    cnx = psycopg2.connect(
        user="citus",
        password=password,
        host="dbcloudeapp.postgres.database.azure.com",
        port=5432,
        database=database
    )

    # Create a cursor
    cursor = cnx.cursor()

    cursor.execute(sql_queue)

    # Commit the transaction
    cnx.commit()

    # Close the cursor and connection
    cursor.close()
    cnx.close()
    

def executeSQL_expectReturn(sql_queue,database): 
    # Establish a connection to the PostgreSQL server with AUTOCOMMIT enabled
    cnx = psycopg2.connect(
        user="citus",
        password=password,
        host="dbcloudeapp.postgres.database.azure.com",
        port=5432,
        database=database
    )

    # Create a cursor
    cursor = cnx.cursor()

    cursor.execute(sql_queue)
    result=cursor.fetchall()

    # Commit the transaction
    cnx.commit()

    # Close the cursor and connection
    cursor.close()
    cnx.close()
    return result
def executeSQL_expectReturn_clean(sql_queue,database): 
    return executeSQL_expectReturn(sql_queue,database)[0][0]

In [44]:
sql_queue = """
    DROP TABLE IF EXISTS account;
    CREATE TABLE account (
        user_id SERIAL PRIMARY KEY,
        username VARCHAR(255) UNIQUE  NOT NULL,
        password VARCHAR(255) NOT NULL,
        role INT NOT NULL
    );
    
    DROP TABLE IF EXISTS user_uploaded_text;
    CREATE TABLE user_uploaded_text (
        text_id SERIAL PRIMARY KEY,
        user_id INT REFERENCES account(user_id),
        uploaded_text TEXT CHECK(LENGTH(uploaded_text) <= 5000),
        upload_timestamp TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );

    """
database="user_content" 
executeSQL(sql_queue,database)

In [3]:
sql_queue= """
CREATE OR REPLACE FUNCTION create_user(
    in_username VARCHAR(255),
    in_password VARCHAR(255),
    in_role INT
)
RETURNS INT AS
$$
DECLARE
    error_message VARCHAR(255);
BEGIN
    -- Check if the username already exists
    IF EXISTS (SELECT 1 FROM account WHERE username = "in_username") THEN
        RETURN 0;
    ELSE
        -- Insert a new user into the "users" table
        INSERT INTO account (username, password, role)
        VALUES (in_username, in_password, in_role);
        RETURN 1;  -- Indicates success
    END IF;
END;
$$
LANGUAGE plpgsql;

CREATE OR REPLACE FUNCTION get_user_role(p_username VARCHAR(255), p_password VARCHAR(255))
RETURNS INT
AS $$
DECLARE
    v_role INT;
BEGIN
    -- Attempt to fetch the role for the given username and password
    SELECT role INTO v_role
    FROM account
    WHERE username = p_username AND password = p_password;

    -- Check if the role is found or not
    IF FOUND THEN
        RETURN v_role;
    ELSE
        -- Return -1 if the credentials are incorrect
        RETURN -1;
    END IF;
END;
$$ LANGUAGE plpgsql;

    """
database="user_content" 
executeSQL(sql_queue,database)


In [76]:
_username="123123123123"
_password="1231232313"
sql_queue = f"""

drop function create_user(
    in_username VARCHAR(255),
    in_password VARCHAR(255),
    in_role INT);
    """
database="user_content" 
a=executeSQL_expectReturn(sql_queue,database)
a[0][0]

In [43]:
import os

# Specify the path to your directory containing text files
directory_path = r'C:\Users\Asus\Downloads\Cranfield'

# Initialize variables to store the longest line and its length
longest_line = ""
max_line_length = 0

# Iterate through all files in the directory
for filename in os.listdir(directory_path):
    if filename.endswith('.txt'):
        file_path = os.path.join(directory_path, filename)

        # Read the single line from the file
        with open(file_path, 'r') as file:
            line = file.readline().strip()

        # Update the longest line if the current line is longer
        if len(line) > max_line_length:
            longest_line = line
            max_line_length = len(line)

# Print the result
print(f"Longest line: {longest_line}")
print(f"Length of the longest line: {max_line_length}")


Longest line: interaction between shock waves and boundary layers, with a note on the effects of the interaction of the performance of supersonic intakes . the interaction between shock waves and boundary layers has important effects in many problems of high speed flow . this paper has been written as a guide to the literature on the subject, and as a critical review of the present state of knowledge concerning both the underlying physical processes and the practical applications . it will be clear to the reader that, although substantial progress has been made, our knowledge is still far from complete and that more work both of a fundamental nature and on specific applications is needed before the problem is understood sufficiently well for design purposes . part i of the paper describes experiments on comparatively simple types of flow designed to provide fundamental information and to assist in the development of the theory . these experiments show that the interaction depends mainl